In [1]:
import os
import torch
import random
import torch.nn as nn
import torch.backends.cudnn as cudnn
import numpy as np
from models import build_model
from PIL import Image
from IPython.display import Image as ipython_image
from diffusers.utils import load_image, export_to_video, export_to_gif

/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.0+cu118 with CUDA 1108 (you have 2.5.1+cu124)
    Python  3.10.11 (you have 3.10.15)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
/home/jovyan/.mlspace/envs/vide

Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'


/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
from conversation import default_conversation, DEFAULT_IMAGE_TOKEN, DEFAULT_VIDEO_TOKEN, VIDEO_TOKEN_INDEX, IMAGE_TOKEN_INDEX
DEFAULT_VISUAL_TOKEN = DEFAULT_VIDEO_TOKEN
VISUAL_TOKEN_INDEX = VIDEO_TOKEN_INDEX

In [3]:
from datasets import load_dataset
dataset_train = load_dataset("Sayankotor/video_dataset_qa",split="train[:80%]")
dataset_valid = load_dataset("Sayankotor/video_dataset_qa",split="train[80%:]")

README.md:   0%|          | 0.00/429 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/64.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/441337 [00:00<?, ? examples/s]

In [4]:
len(dataset_valid)

88267

In [6]:
ink =0

for elem in dataset_valid:
    if (os.path.isfile(elem['video_path'])!= True):
        print(elem['video_path'])
        ink+=1

ink 

0

In [7]:
from models.modeling_video_lavit_hf import VideoLaVITLlamaForCausalLM
from models.transform import LaVITImageProcessor, LaVITEvalVideoProcessor


In [8]:
import transformers
img_size=224
model_dtype="bf16",
apply_lemmatizer=True
use_xformers=False
max_frames=24
max_clips=8
motion_vocab_size=1026
visual_vocab_size = 16384

device_map={"": 0}

model_path = "/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft"#"/home/jinyang06/models/VideoLaVIT-v1/language_model_sft"
model_dtype='bf16'
config = transformers.AutoConfig.from_pretrained(model_path)
config.use_xformers = use_xformers

# For inference, we should use the left padding
config.tokenizer_padding_side = 'left'
config.use_cache = True

tokenizer = transformers.LlamaTokenizer.from_pretrained(model_path, use_fast=False, padding_side='left')
tokenizer.pad_token = tokenizer.unk_token

model = VideoLaVITLlamaForCausalLM.from_pretrained(model_path, config=config, device_map=device_map,
                torch_dtype=torch.bfloat16 if model_dtype=="bf16" else torch.float16,)

for name, param in model.named_parameters():
    param.requires_grad = False

model.eval()

visual_vocab_size = visual_vocab_size


Not used {}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/modeling_utils.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkp

In [9]:
from models.transform import LaVITImageProcessor, LaVITEvalVideoProcessor
video_processor = LaVITEvalVideoProcessor(image_size=224, num_frames=24, fps=6, max_clips=8,)

In [10]:
!pwd

/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT


In [11]:
## Dataset / collator

In [12]:
def tokenize_function(examples):
    tokens = tokenizer(examples['text'], max_length = model.config.max_position_embeddings, truncation=True)
    tokens['labels'] = tokens['input_ids'][len(tokens['input_ids'])//2:]
    tokens['attention_mask'] = tokens['attention_mask'][:len(tokens['input_ids'])//2]
    tokens['input_ids'] = tokens['input_ids'][:len(tokens['input_ids'])//2]
    return tokens 

#train_dataset = ds['train'].map(#raw_datasets['validation'].map(
#    tokenize_function,
#    desc="Running tokenizer on dataset",
#    remove_columns=ds['train'].column_names
#)

## pl training

In [13]:
import pytorch_lightning as pl

In [14]:
from models.video_lavit_for_understanding import tokenizer_image_token

In [15]:
def collate_fn(batch):
    return batch

In [16]:
tokenizer.pad_token_id

0

In [17]:
import os
cwd = os.getcwd()

class VideoLaVIT_pl(pl.LightningModule):
    def __init__(self, video_processor, model, train_dataset, valid_dataset, tokenizer, collate_function):
        super(VideoLaVIT_pl, self).__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.train_dataset = train_dataset
        self.valid_dataset = valid_dataset
        self.video_processor = video_processor
        #self.n_embeddings = model.model.embed_tokens.weight.shape[0]
        self.loss = nn.CrossEntropyLoss(reduction="none", ignore_index=tokenizer.pad_token_id)
        self.train_dataset = train_dataset
        self.validation_dataset = dataset_valid
        self.collate_function = collate_function
        self.n_iters = 100000# len(self.train_dataloader())
        self.n_warmup = 5000
        self.grad_clip = 1.0
        self.weight_decay = 0.001
        self.grad_accum = 512
        self.lr = 0.0001
        self.beta1 = 0.9
        self.beta2 = 0.99
        # self.automatic_optimization = False
        
        
    def configure_optimizers(self):
        
        optimizer = AdamW(list(self.model.model.visual_tokenizer.parameters()) + list(self.model.model.motion_tokenizer.parameters()), lr=self.learning_rate, relative_step=False)
        scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=self.n_warmup, num_training_steps=self.n_iters // self.cfg.grad_accum, betas =(self.beta1, self.beta2), weight_decay = self.weight_decay)
        return {'optimizer': optimizer, 'lr_scheduler': {
            'scheduler': scheduler,
            'interval': 'step',
            'frequency': 1
            
        }}
    

    def on_train_epoch_end(self):
        print ("epoch end")
        #torch.save(self.model.visual_tokenizer, cwd + f"custom_ckpts/visual_tokenizer")
        #torch.save(self.model.motion_tokenizer, cwd+ f"custom_ckpts/motion_tokenizer")

        
    def training_step(self, batch, batch_idx):
        question = batch[0]['question']
        answer = batch[0]['answer']
        video_path = batch[0]['video_path']
        visual_inputs, motion_inputs = self.video_processor(video_path)
        imgs = [([visual_inputs, motion_inputs], 'video')]

        text_inputs = [DEFAULT_VISUAL_TOKEN + "\n" + text_input for text_input in [question ]]
        
        prompts = []
        for text_input in text_inputs:
            conv = default_conversation.copy()
            conv.append_message(conv.roles[0], text_input)
            conv.append_message(conv.roles[1], None)
            prompt = conv.get_prompt()
            prompts.append(prompt)

        print("prompts", prompts)

        input_ids = [tokenizer_image_token(prompt, tokenizer, VISUAL_TOKEN_INDEX, DEFAULT_VISUAL_TOKEN, return_tensors='pt') for prompt in prompts]

        print("input_ids:", input_ids)
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids,
            batch_first=True,
            padding_value=tokenizer.pad_token_id,
        ).to(self.model.device)

        attention_mask = input_ids.ne(tokenizer.pad_token_id)

        labels = [tokenizer(answer, return_tensors='pt').input_ids[0] for answer in answers]

        print ("answer", answer)
        print ("labels", labels)

        position_ids = torch.arange(0, input_ids.shape[1], dtype=torch.long, device=input_ids.device)
        
        with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
            logits = self.model(input_ids = input_ids, attention_mask = attention_mask, position_ids = position_ids, past_key_values = None, inputs_embeds = None, labels = None, images = imgs).logits

        print ("logits.shape, labels.shape", logits.shape, labels.shape)
        #labels = labels[:, 1:]
        #mask = mask[:, 1:]
          
        #logits = logits[mask].contiguous().float()
        #labels = labels[mask].contiguous()

        
        loss = self.loss(logits, labels).mean()
            
        self.log("my_loss", loss, on_step=True, on_epoch=True, prog_bar=True, sync_dist=True)
        return loss

    def validation_step(self, batch, batch_idx):
        #print (batch[0]['question'])
        question = batch[0]['question']
        answers = [batch[0]['answer']]
        video_path = batch[0]['video_path']
        try:
            visual_inputs, motion_inputs = self.video_processor(video_path)
        except:
            print ("Exeption in loading", video_path)
        pass
        imgs = [([visual_inputs, motion_inputs], 'video')]
        

        text_inputs = [DEFAULT_VISUAL_TOKEN + "\n" + text_input for text_input in [question ]]
        
        prompts = []

        for text_input in text_inputs:
            conv = default_conversation.copy()
            conv.append_message(conv.roles[0], text_input)
            conv.append_message(conv.roles[1], None)
            prompt = conv.get_prompt()
            prompts.append(prompt)

        #print("prompts", prompts)

        input_ids = [tokenizer_image_token(prompt, tokenizer, VISUAL_TOKEN_INDEX, DEFAULT_VISUAL_TOKEN, return_tensors='pt') for prompt in prompts]

        

        #print (answers)

        labels = [tokenizer(answer, return_tensors='pt').input_ids[0] for answer in answers]

        ids_with_labels = [torch.cat((input_id, label), 0) for input_id, label in zip(input_ids, labels)]

        

        ids_with_labels = torch.nn.utils.rnn.pad_sequence(ids_with_labels, batch_first=True, padding_value=tokenizer.pad_token_id).to(self.model.device)

        len_labels = [len(label) for label in labels] 

        attention_mask = ids_with_labels.ne(tokenizer.pad_token_id)

        #print ("answer", answers)
        #print ("labels", labels)
        #print ("attention_mask", attention_mask)
        
        
        with torch.no_grad():
            logits = self.model(input_ids = ids_with_labels, attention_mask = attention_mask, past_key_values = None, inputs_embeds = None, labels = None, images = imgs).logits

        loss_mask = torch.zeros(logits.shape, device=self.model.device).bool()
        for i, cur_len in enumerate(len_labels):
            loss_mask[i, -cur_len:, ...] = True

        labels = torch.stack(labels, dim=0).to(model.device)
        #print ("labels2", labels)
        

        #print (loss_mask.shape)
        #print ("logits.shape", logits.shape)
        #print ("mask.shape", loss_mask.shape)
        new_logits = logits[loss_mask].reshape((logits.shape[0], logits.shape[2],  -1))

        #labels = labels[:, 1:]
        #new_logits = new_logits[:, 1:]

        #print ("new logits.shape, labels.shape", new_logits.shape, labels.shape)
        #labels = labels[:, 1:]
        #mask = mask[:, 1:]
          
        #logits = logits[mask].contiguous().float()
        #labels = labels[mask].contiguous()

        
        loss = self.loss(new_logits.contiguous().float(), labels.contiguous()).mean()
        self.log("val_loss", loss, on_step=True, batch_size=1, on_epoch=True, prog_bar=True)
        

In [18]:
module = VideoLaVIT_pl(video_processor, model, dataset_train, dataset_valid, tokenizer, collate_fn)


In [19]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset_train, batch_size=1, collate_fn=collate_fn, num_workers = 1)

val_dataloader = DataLoader(dataset_valid, batch_size=1, collate_fn=collate_fn, num_workers = 1)


In [20]:
from pytorch_lightning.loggers import CSVLogger
logger = CSVLogger("ckpts", name="validation")

In [21]:
trainer = pl.Trainer(devices=1, logger =logger, log_every_n_steps=50)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
trainer.validate(model=module, dataloaders=val_dataloader)

You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=255` in the `DataLoader` to improve performance.


Validation: |          | 0/? [00:00<?, ?it/s]

/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/modeling_motion_tokenizer.py:433: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
[m4v @ 0xe7b1b40] Format m4v detected only with low score of 25, misdetection possible!
[m4v @ 0x7f38a5a62800] Format m4v detected only with low score of 25, misdetection possible!


Exeption in loading /home/jovyan/shares/SR004.nfs2/data/LLaVA-Video-178K/LLaVA-Video-178K/1_2_m_academic_v0_1/academic_source/activitynet/v_AOBkrb8yYS4.mp4


UnboundLocalError: local variable 'visual_inputs' referenced before assignment

In [25]:
labels

NameError: name 'labels' is not defined